In [2]:
"""
YouTube Transcript Retrieval Script
Created by: Hamed Ghane
Date: December 06, 2024

This script retrieves a transcript from a YouTube video using the YouTube Transcript API.
It performs the following steps:
1. Extract the video ID from a given YouTube URL.
2. Use the YouTube Transcript API to request the transcript of that video.
3. Print the transcript to the console if available.

Requirements:
- youtube_transcript_api (Install via: pip install youtube-transcript-api)

Usage:
- Run this script.
- When prompted, enter a valid YouTube URL.
- If a transcript is available, it will be printed to the console.
- If no transcript is available or the URL is invalid, an error message will be displayed.
"""

import re  # Used for regular expressions to extract the video ID from the URL
from youtube_transcript_api import YouTubeTranscriptApi  # Used to fetch YouTube transcripts

def get_video_id(url):
    """
    Extract the YouTube video ID from a given URL using a regular expression.
    Returns the video ID if found, otherwise returns None.
    """
    # Search the provided URL for a pattern that matches a YouTube video ID.
    # The pattern looks for 'v=' or '/' followed by 11 valid characters typical of a YouTube video ID.
    video_id_match = re.search(r"(?:v=|\/)([0-9A-Za-z_-]{11}).*", url)
    
    # If a match is found, return the captured video ID group.
    if video_id_match:
        return video_id_match.group(1)
    
    # If no match is found, return None indicating an invalid or non-matching URL.
    return None

def get_transcript(video_id):
    """
    Retrieve the transcript for the given YouTube video ID using the YouTube Transcript API.
    If successful, returns the transcript as a string. Otherwise, returns None.
    """
    try:
        # Fetch the transcript data from YouTube using the video ID.
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        
        # The transcript is a list of dictionaries with 'text' keys. 
        # Join all text entries into a single string separated by spaces.
        return " ".join([entry['text'] for entry in transcript])
    
    except Exception as e:
        # If an error occurs (e.g., no transcript found, network issue), print the error and return None.
        print(f"An error occurred: {e}")
        return None

def main():
    """
    Main function of the script:
    1. Prompts the user to input a YouTube URL.
    2. Extracts the video ID from the URL.
    3. Attempts to retrieve the transcript using the obtained video ID.
    4. Prints the transcript if available, or shows an error message otherwise.
    """
    # Prompt the user to enter a YouTube URL.
    url = input("Enter the YouTube URL: ")
    
    # Extract the video ID from the provided URL.
    video_id = get_video_id(url)
    
    # If no valid video ID could be extracted, print an error and stop execution.
    if not video_id:
        print("Invalid YouTube URL")
        return
    
    # Attempt to retrieve the transcript using the extracted video ID.
    transcript = get_transcript(video_id)
    
    # If no transcript could be retrieved, notify the user and stop execution.
    if not transcript:
        print("Failed to retrieve transcript")
        return
    
    # If the transcript retrieval is successful, print it to the console.
    print("\nFull Transcript:")
    print(transcript)

# Run the main function if this file is executed as the main script.
if __name__ == "__main__":
    main()


Enter the YouTube URL:  https://www.youtube.com/shorts/sMtpi3cchNQ



Full Transcript:
here is a hot beverage to comfort you it's in a tog go cup make of that what you will come on it's still early let's do something well I have been toying around with an idea for 4 d chest how about we just talk all right in 4 d chest no come on let's talk about our lives tell me something about you I don't know I own nine pairs of pants okay that that's a good start but I was thinking maybe something a little more personal I see I own nine pairs of Underpants how about I go first but I don't want to know how many Underpants you own although base on the floor of your bedroom I'd say it's a thousand


In [ ]:

"""
YouTube Video Transcription GUI Application

Created by: Hamed Ghane
Date: December 06, 2024

This script creates a GUI application that:
1. Takes a YouTube URL from the user.
2. Attempts to retrieve the video's transcript from YouTube directly.
3. If no transcript is found, it downloads the video's audio and uses OpenAI's Whisper model to transcribe it.
4. Allows the user to choose between Persian or English transcription.
5. Displays the transcript in the GUI with progress and status updates.

Requirements:
- PyQt5 for the GUI (pip install PyQt5)
- openai for Whisper transcription (pip install openai)
- youtube_transcript_api to fetch existing transcripts (pip install youtube-transcript-api)
- yt-dlp to download audio (pip install yt-dlp)
- A file named 'sk.py' containing `Hamedkey = "<YOUR_OPENAI_API_KEY>"` or replace the import with your key directly.

The script also logs its actions to a log file in the user's Documents folder and prints logs to the console.

Usage:
- Run this script.
- Enter the YouTube URL in the provided text box.
- Select the desired language.
- Click "Transcribe" to retrieve or generate the transcript.
"""

import sys  # For system-related operations and command-line arguments
import os   # For operating system dependent functionality (like paths)
import openai  # Interact with OpenAI Whisper API for transcription
import logging  # For logging information, warnings, and errors
from pathlib import Path  # For convenient file path handling
from PyQt5.QtWidgets import (QApplication, QMainWindow, QPushButton, QVBoxLayout, 
                             QWidget, QLabel, QTextEdit, QProgressBar, QLineEdit, 
                             QHBoxLayout, QComboBox, QMessageBox)
from PyQt5.QtCore import Qt, QTimer
from sk import Hamedkey  # Import your OpenAI API key from sk.py
import re  # Regular expressions for extracting video ID from a URL
from youtube_transcript_api import YouTubeTranscriptApi  # Fetch transcripts from YouTube
import subprocess  # For running external commands (yt-dlp)
import time  # To implement retry logic with waiting periods
from typing import Optional  # For type hinting optional returns

# Set up a log file in the user's Documents folder to avoid permission issues
log_path = os.path.join(os.path.expanduser('~'), 'Documents', 'transcription_app.log')
logging.basicConfig(
    level=logging.INFO,  # Log info, warnings, and errors
    format='%(asctime)s - %(levelname)s - %(message)s',  # Define the log format
    handlers=[
        logging.FileHandler(log_path),  # Write logs to a file
        logging.StreamHandler()         # Also print logs to the console
    ]
)

def check_dependencies() -> None:
    """
    Verify that required dependencies (openai, youtube_transcript_api, PyQt5, yt-dlp) are installed and accessible.
    Raises ImportError if any are missing.
    """
    # Essential packages to check
    essential_packages = ['openai', 'youtube_transcript_api', 'PyQt5']
    missing = []
    
    # Check Python packages
    for package in essential_packages:
        try:
            __import__(package)  # Attempt to import the package
        except ImportError:
            missing.append(package)
    
    # Special check for yt-dlp since it's an external tool
    try:
        # Try running yt-dlp --version quietly
        subprocess.run(['yt-dlp', '--version'], 
                       capture_output=True, 
                       text=True, 
                       creationflags=subprocess.CREATE_NO_WINDOW)
    except FileNotFoundError:
        # If yt-dlp is not found at all
        missing.append('yt-dlp')
    except Exception as e:
        # Any other error during checking yt-dlp
        logging.warning(f"yt-dlp check error: {e}")
        missing.append('yt-dlp')
    
    # If anything is missing, raise an error
    if missing:
        error_msg = f"Missing required packages. Please install: {', '.join(missing)}"
        logging.error(error_msg)
        raise ImportError(error_msg)

def get_video_id(url: str) -> Optional[str]:
    """
    Extract the YouTube video ID from the provided URL using a regex pattern.
    Returns the video ID if found, otherwise None.
    """
    # Regex looks for 'v=' or '/' followed by 11 valid YouTube ID characters
    video_id_match = re.search(r"(?:v=|\/)([0-9A-Za-z_-]{11}).*", url)
    # If a match is found, return the group (the video ID)
    return video_id_match.group(1) if video_id_match else None

def download_audio_from_youtube(video_id: str, progress_callback=None) -> Optional[str]:
    """
    Download the YouTube video's audio track using yt-dlp.
    If progress_callback is provided, it's called with the download percentage.
    Returns the path to the downloaded .mp3 file if successful, otherwise None.
    """
    try:
        # Save the downloaded audio in the user's Documents folder
        output_path = os.path.join(os.path.expanduser('~'), 'Documents', 'downloaded_audio.%(ext)s')
        
        # Command to run yt-dlp to download and convert the video to MP3 audio
        command = [
            'yt-dlp',
            '--extract-audio',
            '--audio-format', 'mp3',
            '-o', output_path,
            '--newline',  # Print progress in new lines
            f'https://www.youtube.com/watch?v={video_id}'
        ]
        
        # Start the subprocess for downloading
        process = subprocess.Popen(
            command,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            universal_newlines=True,
            creationflags=subprocess.CREATE_NO_WINDOW
        )
        
        # Continuously read lines from yt-dlp's stdout
        while True:
            output = process.stdout.readline()
            # If no more output and the process ended, break out of loop
            if output == '' and process.poll() is not None:
                break
            if output and progress_callback:
                # Lines containing '[download]' and '%' indicate download progress
                if '[download]' in output and '%' in output:
                    try:
                        # Attempt to parse the percentage
                        percent = float(output.split('%')[0].split()[-1])
                        progress_callback(percent)
                    except (ValueError, IndexError):
                        # If parsing fails, just ignore and continue
                        pass
        
        # If process finished successfully
        if process.returncode == 0:
            # Return the path of the downloaded file (yt-dlp uses 'downloaded_audio.mp3')
            return os.path.join(os.path.expanduser('~'), 'Documents', 'downloaded_audio.mp3')
        else:
            # If not successful, read the error output and log it
            error_output = process.stderr.read()
            logging.error(f"Download error: {error_output}")
            return None
            
    except Exception as e:
        # Log and return None if any exception occurs
        logging.error(f"Error downloading audio: {e}")
        return None

def fetch_youtube_transcript(video_id: str, language: str) -> Optional[str]:
    """
    Attempt to fetch an existing YouTube transcript for the given video ID.
    The language parameter is "Persian" or "English".
    Returns the transcript text if found, otherwise None.
    """
    try:
        # Get the list of transcripts for the given video
        transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)
        
        # Determine the language code from the chosen language
        lang_code = 'fa' if language == 'Persian' else 'en'
        
        try:
            # Try finding a transcript in the requested language code
            selected_transcript = transcript_list.find_transcript([lang_code])
        except:
            # If that fails, try any manually created transcript for English or Persian
            # If none found, try a generated one
            selected_transcript = transcript_list.find_manually_created_transcript(['en', 'fa']) \
                                or transcript_list.find_generated_transcript(['en', 'fa'])
        
        # Fetch the transcript data and join all text entries
        transcript_data = selected_transcript.fetch()
        return " ".join([entry['text'] for entry in transcript_data])
    
    except Exception as e:
        # Log error if fetching fails
        logging.error(f"Error fetching YouTube transcript: {e}")
        return None

def whisper_transcribe(audio_file_path: str, language: str, max_retries: int = 3) -> str:
    """
    Transcribe the given audio file using the OpenAI Whisper API.
    Retries up to max_retries times if rate-limited.
    
    language: "Persian" or "English" - we map this to 'fa' or 'en'.
    Returns the transcript text if successful.
    Raises an exception if transcription fails after all attempts.
    """
    # Map language to appropriate code for Whisper
    lang_code = 'fa' if language == 'Persian' else 'en'
    
    # Set the OpenAI API key
    openai.api_key = Hamedkey

    # Attempt transcription multiple times in case of rate limit errors
    for attempt in range(max_retries):
        try:
            # Open the audio file in binary mode
            with open(audio_file_path, "rb") as audio:
                # Send request to the Whisper model
                transcript = openai.Audio.transcribe(
                    model="whisper-1",
                    file=audio,
                    language=lang_code,
                    response_format="text"
                )
            # If successful, return the transcript
            return transcript
        except openai.error.RateLimitError:
            # If rate-limited, wait and retry
            if attempt < max_retries - 1:
                wait_time = 2 ** attempt  # Exponential backoff
                time.sleep(wait_time)
            else:
                # If last attempt failed, re-raise the error
                raise
        except Exception as e:
            # Log other errors and if last attempt, raise the error
            logging.error(f"Transcription error on attempt {attempt + 1}: {e}")
            if attempt == max_retries - 1:
                raise

def cleanup_audio_file(audio_file_path: str) -> None:
    """
    Delete the temporary audio file after transcription is done.
    Logs a warning if deletion fails.
    """
    try:
        Path(audio_file_path).unlink(missing_ok=True)
        logging.info(f"Cleaned up audio file: {audio_file_path}")
    except Exception as e:
        logging.warning(f"Could not delete temporary audio file: {e}")

def create_window():
    """
    Create and configure the main application window and all its widgets:
    - Input fields for YouTube URL
    - Language selection
    - Labels for status and instructions
    - A progress bar for download/transcription progress
    - A text area to display the final transcript
    - A button to start the transcription process

    Returns the window object.
    """
    # Create the main application window
    window = QMainWindow()
    window.setWindowTitle('YouTube Audio Transcription')  # Set window title
    window.setGeometry(100, 100, 800, 600)  # Set window size
    
    # Create a central widget to hold the layout and other widgets
    central_widget = QWidget()
    window.setCentralWidget(central_widget)
    layout = QVBoxLayout(central_widget)  # Vertical layout
    
    # Create a welcome label at the top
    welcome_label = QLabel('YouTube Audio Transcription')
    welcome_label.setAlignment(Qt.AlignCenter)  # Center the text
    welcome_label.setStyleSheet('font-size: 16px; margin: 10px; font-weight: bold;')
    layout.addWidget(welcome_label)
    
    # Create a horizontal layout for URL input and language selection
    input_layout = QHBoxLayout()
    url_input = QLineEdit()
    url_input.setPlaceholderText("Enter YouTube URL here...")  # Guide the user
    input_layout.addWidget(url_input)
    
    # Create a combo box for language choice: English or Persian
    language_combo = QComboBox()
    language_combo.addItems(["English", "Persian"])
    language_combo.setToolTip("Select transcription language")
    input_layout.addWidget(language_combo)
    layout.addLayout(input_layout)
    
    # A label to show the current status or instructions
    status_label = QLabel('')
    status_label.setAlignment(Qt.AlignCenter)
    layout.addWidget(status_label)
    
    # A progress bar to indicate downloading/transcribing progress
    progress_bar = QProgressBar()
    progress_bar.setRange(0, 100)  # 0 to 100% for downloads
    progress_bar.hide()  # Hidden by default
    layout.addWidget(progress_bar)
    
    # A text area to display the transcript result
    transcription_text = QTextEdit()
    transcription_text.setPlaceholderText("Transcription will appear here...")
    transcription_text.setReadOnly(True)
    layout.addWidget(transcription_text)
    
    # A button to start the transcription
    transcribe_btn = QPushButton('Transcribe')
    # Style the button
    transcribe_btn.setStyleSheet('''
        QPushButton {
            padding: 10px;
            font-size: 14px;
            background-color: #4CAF50;
            color: white;
            border: none;
            border-radius: 5px;
            min-width: 150px;
        }
        QPushButton:hover {
            background-color: #45a049;
        }
        QPushButton:disabled {
            background-color: #cccccc;
            color: #666666;
        }
    ''')
    layout.addWidget(transcribe_btn, alignment=Qt.AlignCenter)
    
    def update_progress(percent):
        """
        Update the progress bar with the given percentage.
        Calls QApplication.processEvents() to ensure UI updates promptly.
        """
        progress_bar.setValue(int(percent))
        QApplication.processEvents()
    
    def display_transcript(text: str) -> None:
        """
        Display the transcript in the text area.
        If the transcript is very long, truncate it and indicate so.
        """
        max_length = 100000  # Arbitrary large limit
        if len(text) > max_length:
            truncated_text = text[:max_length] + "\n\n[Transcript truncated...]"
            transcription_text.setText(truncated_text)
        else:
            transcription_text.setText(text)
    
    def start_transcription():
        """
        Validate the input URL and initiate the transcription process.
        Shows errors if URL is invalid. Otherwise, start processing.
        """
        url = url_input.text().strip()
        language = language_combo.currentText()
        
        # Check if URL is provided
        if not url:
            status_label.setText("Please enter a YouTube URL")
            status_label.setStyleSheet('color: red;')
            return
        
        # Extract video ID
        video_id = get_video_id(url)
        if not video_id:
            status_label.setText("Invalid YouTube URL")
            status_label.setStyleSheet('color: red;')
            return
        
        # Disable the transcribe button and reset progress bar
        transcribe_btn.setEnabled(False)
        progress_bar.setValue(0)
        progress_bar.show()
        
        # Use QTimer to avoid freezing the UI before starting the work
        QTimer.singleShot(100, lambda: process_transcription(video_id, language))
    
    def process_transcription(video_id: str, language: str):
        """
        The main logic for handling transcription:
        1. Try to get YouTube transcript directly.
        2. If not available, download audio and use Whisper.
        3. Show progress and handle cleanup.
        """
        audio_file = None
        try:
            # First, try fetching the YouTube transcript
            status_label.setText("Checking for YouTube transcript...")
            status_label.setStyleSheet('color: blue;')
            QApplication.processEvents()
            
            yt_transcript = fetch_youtube_transcript(video_id, language)
            
            if yt_transcript and yt_transcript.strip():
                # If we got a transcript directly from YouTube
                display_transcript(yt_transcript)
                status_label.setText("YouTube transcript retrieved successfully!")
                status_label.setStyleSheet('color: green;')
            else:
                # No transcript from YouTube, proceed to audio download
                status_label.setText("Downloading audio...")
                progress_bar.setValue(0)
                QApplication.processEvents()
                
                audio_file = download_audio_from_youtube(video_id, update_progress)
                
                if not audio_file:
                    # If we fail to download audio, show error and stop
                    status_label.setText("Failed to download audio")
                    status_label.setStyleSheet('color: red;')
                    return
                
                # Once audio is downloaded, transcribe using Whisper
                status_label.setText("Transcribing with Whisper...")
                # Set progress bar to indefinite mode for transcription phase
                progress_bar.setRange(0, 0)
                QApplication.processEvents()
                
                # Perform Whisper transcription
                transcript = whisper_transcribe(audio_file, language)
                display_transcript(transcript)
                status_label.setText("Transcription completed!")
                status_label.setStyleSheet('color: green;')
                
        except Exception as e:
            # If any exception occurs, log and show error message
            error_msg = str(e)
            logging.error(f"Transcription error: {error_msg}")
            status_label.setText(f"Error: {error_msg}")
            status_label.setStyleSheet('color: red;')
            
        finally:
            # Hide the progress bar, re-enable the button, and clean up audio file if it was downloaded
            progress_bar.hide()
            transcribe_btn.setEnabled(True)
            if audio_file:
                cleanup_audio_file(audio_file)
    
    # Connect the transcribe button to the start_transcription function
    transcribe_btn.clicked.connect(start_transcription)
    
    # Return the configured window
    return window

def main():
    """
    Main function that:
    1. Checks dependencies.
    2. Creates the application and main window.
    3. Runs the application event loop.
    If dependencies are missing or errors occur, shows a message box and exits.
    """
    try:
        # Check if all dependencies are present
        check_dependencies()
        
        # Create the Qt application
        app = QApplication(sys.argv)
        
        # Create and show the main window
        window = create_window()
        window.show()
        
        # Run the application event loop
        sys.exit(app.exec_())
    except ImportError as e:
        # If dependencies are missing, show an error and exit
        QMessageBox.critical(None, "Missing Dependencies", str(e))
        sys.exit(1)
    except Exception as e:
        # Any unexpected error
        QMessageBox.critical(None, "Error", f"An unexpected error occurred: {str(e)}")
        logging.error(f"Application error: {e}")
        sys.exit(1)

# If this script is run directly, execute the main function
if __name__ == '__main__':
    main()